## 获取覆盖率文件

In [11]:
import re, os, sys
import subprocess
from pathlib import Path
import configparser
from tqdm import tqdm

In [12]:
# 获取项目的绝对路径
BASE_DIR = str(Path().resolve().parent.parent)
sys.path.append(BASE_DIR)
print(BASE_DIR)
# 加载配置文件
cf = configparser.ConfigParser()
cf.read(os.path.join(BASE_DIR, "workspace/engineCoverage/coverageCombine.ini"), encoding='utf-8')
# print(cf.get('coverageCombine', 'coverage'))

/root/comModel


['/root/comModel/workspace/engineCoverage/coverageCombine.ini']

In [13]:
def recover_filename(model_testcase_path):
    '''
    恢复js文件名称,需要传入js文件所在的目录
    jupyter中主要目标是进行可机动计算，所以不需要自动化，规定路径即可
    '''

    def recover_filter(f):
        if f[-8:] in ['.changed']:
            return True
        else:
            return False

    recover_testcase_path = filter(recover_filter, os.listdir(model_testcase_path))
    for file in tqdm(list(recover_testcase_path), desc="正在恢复js文件名称: "):
        cmd_coverage = f"mv {os.path.join(model_testcase_path, file)} {os.path.join(model_testcase_path, file[:-8])}"
        pro = subprocess.Popen(cmd_coverage, stdin=subprocess.PIPE, stdout=subprocess.PIPE, shell=True,
                               stderr=subprocess.PIPE, universal_newlines=True)
        pro.communicate()

In [14]:
def getCoverageProfraw(testcases_path):
    '''
        用插桩之后的引擎获取覆盖率文件
    '''
    coverage_path = os.path.join(BASE_DIR, cf.get('coverageCombine', 'coverage'))
    engine_path = os.path.join(BASE_DIR, cf.get('coverageCombine', 'engine'))
    init_path = testcases_path.split("/")
    model_name = init_path[-3]
    model_checkpoint = init_path[-2]
    js_name_list = os.path.join(BASE_DIR, testcases_path)

    # print(path_data)
    js_list = [os.path.join(BASE_DIR, testcases_path, js_name) for js_name in os.listdir(js_name_list) if
               js_name[-3:] == ".js"]

    for js_path in tqdm(js_list, desc="获取js的引擎覆盖率文件: "):
        LLVM_PROFILE_FILE = os.path.join(BASE_DIR, coverage_path, model_name, model_checkpoint,
                                         js_path.split("/")[-1] + ".profraw")
        LLVM_PROFDATA_FILE = os.path.join(BASE_DIR, coverage_path, model_name, model_checkpoint,
                                          js_path.split("/")[-1] + ".profdata")

        cmd = ["timeout", "-s9", "30s", engine_path, js_path]
        my_env = os.environ.copy()
        my_env['LLVM_PROFILE_FILE'] = LLVM_PROFILE_FILE
        # print(LLVM_PROFILE_FILE)
        # print(cmd)
        pro = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, shell=False, env=my_env,
                               stderr=subprocess.PIPE, universal_newlines=True)
        pro.communicate()

        js_change_cmd = "mv " + js_path + " " + js_path + ".changed"
        pro = subprocess.Popen(js_change_cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, shell=True,
                               stderr=subprocess.PIPE, universal_newlines=True)
        # print(js_change_cmd)
        pro.communicate()

        cmd_coverage = "llvm-profdata-10 merge -o " + LLVM_PROFDATA_FILE + " " + LLVM_PROFILE_FILE + " && rm " + LLVM_PROFILE_FILE
        # print(cmd_coverage)
        pro = subprocess.Popen(cmd_coverage, stdin=subprocess.PIPE, stdout=subprocess.PIPE, shell=True,
                               stderr=subprocess.PIPE, universal_newlines=True)
        pro.communicate()


def profrawToProfdata(profraw_path):
    '''
    主要作为验证是否都转为profdata
    '''

    def recover_filter(f):
        if f.split(".")[-1] in ['profraw']:
            return True
        else:
            return False

    profraw_path_list = filter(recover_filter, os.listdir(profraw_path))

    profraw_path_list = [os.path.join(profraw_path, profraw_name) for profraw_name in profraw_path_list]

    for profraw_name_path in tqdm(profraw_path_list, desc="最终检查profraw是否转化完毕:"):
        cmd_coverage = "llvm-profdata-10 merge -o " + profraw_name_path.replace("profraw",
                                                                                "profdata") + " " + profraw_name_path + " && rm " + profraw_name_path
        # print(cmd_coverage)
        pro = subprocess.Popen(cmd_coverage, stdin=subprocess.PIPE, stdout=subprocess.PIPE, shell=True,
                               stderr=subprocess.PIPE, universal_newlines=True)
        # pro.communicate()

In [15]:
def combinProfdata(profdata_path):
    aim_profdata = profdata_path.split("/")[-2] + "_" + profdata_path.split("/")[-1] + ".profdata"

    def recover_filter(f):
        if f.split(".")[-1] in ['profdata'] and f.split(".")[-2] in ['js']:
            return True
        else:
            return False

    profdata_path_list = filter(recover_filter, os.listdir(profdata_path))
    profdata_path_list = [os.path.join(profdata_path, profraw_name) for profraw_name in profdata_path_list]
    if not os.path.exists(os.path.join(profdata_path, aim_profdata)) and len(profdata_path_list) >= 2:
        init_cmd = "llvm-profdata-10 merge -sparse {p0} {p1} -o {Paim} && rm {p0} {p1}".format(
            p0=profdata_path_list[0], p1=profdata_path_list[1], Paim=os.path.join(profdata_path, aim_profdata)
        )
        pro = subprocess.Popen(init_cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, shell=True,
                               stderr=subprocess.PIPE, universal_newlines=True)
        # print(init_cmd)
        pro.communicate()

    for profdata_name_path in tqdm(profdata_path_list, desc="合并profdata:"):
        combine_cmd = "llvm-profdata-10 merge -sparse {Paim} {p0} -o {Paim} && rm {p0}".format(
                            Paim=os.path.join(profdata_path, aim_profdata), p0=profdata_name_path
                        )
        pro = subprocess.Popen(combine_cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, shell=True,
                           stderr=subprocess.PIPE, universal_newlines=True)
        pro.communicate()



In [16]:
# 恢复js name
model_testcase_path = "/root/comModel/data/Finetune_model/Finetune_distilgpt2/checkpoint-10000/testcases"
profraw_path = "/root/comModel/data/coverage/Finetune_distilgpt2/checkpoint-10000"
profrdata_path = "/root/comModel/data/coverage/Finetune_distilgpt2/checkpoint-10000"
if not os.path.exists(profraw_path):
    os.makedirs(profraw_path)
# 恢复js文件
recover_filename(model_testcase_path)
# 获取js的覆盖率文件(profraw)
getCoverageProfraw(model_testcase_path)
# 将profraw转为profdata
profrawToProfdata(profraw_path)
# 覆盖率合并
combinProfdata(profrdata_path)

获取js的引擎覆盖率文件: 100%|██████████| 28/28 [00:41<00:00,  1.50s/it]
最终检查profraw是否转化完毕:: 0it [00:00, ?it/s]
合并profdata:: 100%|██████████| 28/28 [00:11<00:00,  2.36it/s]


## 详细分析覆盖率文件

In [22]:
def getShowProfraw(profraw_combine):
    engine_path = os.path.join(BASE_DIR, cf.get('coverageCombine', 'engine'))
    combine_cmd = "llvm-cov-10 show {engine} -instr-profile={profraw_combine}".format(
                                engine=engine_path, profraw_combine=profraw_combine)
    pro = subprocess.Popen(combine_cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, shell=True,
                       stderr=subprocess.PIPE, universal_newlines=True)
    stdout, stderr = pro.communicate()
    return stdout,stderr
profraw_combine = "/root/comModel/data/coverage/Finetune_distilgpt2/checkpoint-10000/Finetune_distilgpt2_checkpoint-10000.profdata"
stdout,stderr = getShowProfraw(profraw_combine)
with open('report/getShowProfraw.txt','w') as f:    #设置文件对象
    f.write(stdout)                 #将字符串写入文件中

In [32]:
def getReportProfraw(profraw_combine):
    engine_path = os.path.join(BASE_DIR, cf.get('coverageCombine', 'engine'))
    combine_cmd = "llvm-cov-10 report {engine} -instr-profile={profraw_combine}".format(
                                engine=engine_path, profraw_combine=profraw_combine)
    pro = subprocess.Popen(combine_cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, shell=True,
                       stderr=subprocess.PIPE, universal_newlines=True)
    stdout, stderr = pro.communicate()
    return stdout,stderr
profraw_combine = "/root/comModel/data/coverage/Finetune_distilgpt2/checkpoint-10000/Finetune_distilgpt2_checkpoint-10000.profdata"
stdout,stderr = getReportProfraw(profraw_combine)
with open('report/getReportProfraw.txt','w') as f:    #设置文件对象
    f.write(stdout)                 #将字符串写入文件中

llvm-cov-10 report /root/comModel/engine/ch -instr-profile=/root/comModel/data/coverage/Finetune_distilgpt2/checkpoint-10000/Finetune_distilgpt2_checkpoint-10000.profdata


In [37]:
import json
def getExportProfraw(profraw_combine):
    engine_path = os.path.join(BASE_DIR, cf.get('coverageCombine', 'engine'))
    combine_cmd = "llvm-cov-10 export {engine} -instr-profile={profraw_combine}".format(
                                engine=engine_path, profraw_combine=profraw_combine)

    pro = subprocess.Popen(combine_cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, shell=True,
                       stderr=subprocess.PIPE, universal_newlines=True)
    stdout, stderr = pro.communicate()

    return stdout,stderr
profraw_combine = "/root/comModel/data/coverage/Finetune_distilgpt2/checkpoint-10000/Finetune_distilgpt2_checkpoint-10000.profdata"
stdout,stderr = getExportProfraw(profraw_combine)

In [39]:
with open("report/profraw_combine.json", 'w') as write_f:
	write_f.write(stdout)